# Human Computer Interaction (mouse curser movement , right click, scroll-up, scroll-down)

In [ ]:
import cv2 #hi
import mediapipe as mp
import pyautogui
import math
import time

# Screen size
screen_width, screen_height = pyautogui.size()

# MediaPipe setup
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7)

# Webcam
cap = cv2.VideoCapture(0)

# Euclidean distance
def euclidean_distance(p1, p2):
    return math.hypot(p2[0] - p1[0], p2[1] - p1[1])

# Right click gesture
def is_right_click(landmarks, width, height, threshold=30):
    thumb_tip = landmarks.landmark[4]
    index_tip = landmarks.landmark[8]
    x1, y1 = int(thumb_tip.x * width), int(thumb_tip.y * height)
    x2, y2 = int(index_tip.x * width), int(index_tip.y * height)
    return euclidean_distance((x1, y1), (x2, y2)) < threshold

# Finger states
def fingers_up(landmarks):
    tips_ids = [8, 12, 16, 20]
    pip_ids = [6, 10, 14, 18]
    return [
        1 if landmarks.landmark[tip].y < landmarks.landmark[pip].y else 0
        for tip, pip in zip(tips_ids, pip_ids)
    ]

# Click and scroll flags
gesture_log = {"right_click": 0, "scroll_up": 0, "scroll_down": 0}
gesture_success = {"right_click": 0, "scroll_up": 0, "scroll_down": 0}
last_click_time = 0
current_action = ""

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    result = hands.process(rgb_frame)
    current_action = ""

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            # Get positions
            index_tip = hand_landmarks.landmark[8]
            middle_tip = hand_landmarks.landmark[12]
            x1, y1 = int(index_tip.x * w), int(index_tip.y * h)
            x2, y2 = int(middle_tip.x * w), int(middle_tip.y * h)

            # Check if 8 and 12 are close
            if euclidean_distance((x1, y1), (x2, y2)) < 40:
                # Control mouse cursor using index finger
                screen_x = int(index_tip.x * screen_width)
                screen_y = int(index_tip.y * screen_height)
                pyautogui.moveTo(screen_x, screen_y)
                current_action = "Mouse Control"

            # Right click
            if is_right_click(hand_landmarks, w, h):
                gesture_log["right_click"] += 1
                current_time = time.time()
                if current_time - last_click_time > 1:
                    pyautogui.leftClick()
                    last_click_time = current_time
                    gesture_success["right_click"] += 1
                    current_action = "Right Click"

            # Scroll Gesture
            fingers = fingers_up(hand_landmarks)
            if fingers.count(1) == 3:  # Exactly 3 fingers up
                if fingers[:3] == [1, 1, 1]:
                    gesture_log["scroll_up"] += 1
                    pyautogui.scroll(100)
                    gesture_success["scroll_up"] += 1
                    current_action = "Scrolling Up"
            elif fingers == [0, 0, 0, 0]:  # All 4 fingers down
                gesture_log["scroll_down"] += 1
                pyautogui.scroll(-100)
                gesture_success["scroll_down"] += 1
                current_action = "Scrolling Down"

            # Draw hand
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Show current action on screen
    if current_action:
        cv2.putText(frame, current_action, (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)

    cv2.imshow("Mouse Control", frame)
    time.sleep(0.05)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()

# Print Accuracy Report
print("\n--- Gesture Accuracy Report ---")
for gesture in gesture_log:
    total = gesture_log[gesture]
    success = gesture_success[gesture]
    accuracy = (success / total * 100) if total > 0 else 0
    print(f"{gesture.title()}:")
    print(f"  Detected: {total}")
    print(f"  Performed: {success}")
    print(f"  Accuracy: {accuracy:.2f}%\n")
